In [ ]:
from model import DANN_nasa, DANN_res152
from dataset import Real, Image, Image_feature
from torch.utils.data import DataLoader
import os
import torch
import random
import numpy as np
import torch.backends.cudnn as cudnn
cudnn.benchmark = True
cuda = True if torch.cuda.is_available() else False

In [ ]:
# function of top k accuarcy
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [ ]:
alpha = 0

batch_size = 256

model_root = 'dann_models'

data_dir = 'dataset'

t_name = 'real'

t_dir = os.path.join(data_dir, t_name)
t_csv = [os.path.join(t_dir, f) for f in os.listdir(t_dir) if f.endswith('test.csv')][0]
print(t_csv)

In [ ]:
os.listdir(os.path.join(model_root,'sketch2real'))

In [ ]:
### key in the models that you want to test ###
#model_list = os.listdir(os.path.join(model_root,'sketch2real'))
model_list = ['sketch2real_resnet152_best_0.42062.pth', 'sketch2real_resnet152_best_0.48847.pth']
###

# list that stores all the output tensor after each pth
out_list = []

# top-1 accuaracy for each pth
top_1 = []

# top-5 accuaracy for each pth
top_5 = []

In [ ]:
for model in model_list:
    
    # define model corresponding to the current testing pth

    if 'resnet152' in model:

        feature_model = 'resnet152'

        net = DANN_res152()

    elif 'nasnetalarge' in model:

        feature_model = 'nasnetalarge'

        net = DANN_nasa()
        
    # build target dataloader

    dataset_t = Image_feature(data_dir, t_csv, feature_model)
    dataloader_t = DataLoader(dataset=dataset_t, batch_size=batch_size, shuffle=False, num_workers=4)

    print('len(dataset_t): ', len(dataset_t))  # total # of target images
    print('len(dataloader_t): ', len(dataloader_t))

    for i, data in enumerate(dataloader_t):
        images, labels = data
        print('Image tensor in each batch:', images.shape, images.dtype)
        print('Label tensor in each batch:', labels.shape, labels.dtype)
        break

    # check pth path
    model_path = os.path.join(model_root, model.split('_')[0], model)
    print(model_path)
    
    # load pth
    net = torch.load(model_path)

    net = net.eval()

    if cuda:
        net = net.cuda()

    len_dataloader = len(dataloader_t)
    data_target_iter = iter(dataloader_t)
    #print('len_dataloader: ', len_dataloader)
    
    i = 0
    n_correct = 0
    n_total = 0
    while i < len_dataloader:

        # test model using target data
        data_target = data_target_iter.next()
        t_img, t_label = data_target

        batch_size = len(t_label)        
        class_label = torch.LongTensor(batch_size)

        if cuda:
            t_img = t_img.cuda()
            t_label = t_label.cuda()            
            class_label = class_label.cuda()

        class_label.resize_as_(t_label).copy_(t_label)
        
        with torch.no_grad():
            class_output, _ = net(input_data=t_img, alpha=alpha)

        if i == 0:
            out_tensor = class_output
            out_label = class_label
        else:
            out_tensor = torch.cat((out_tensor, class_output), 0)
            out_label = torch.cat((out_label, class_label), 0)

        if i % (len_dataloader//5) == 0:
            print('progress: %d%% ' % (i/len_dataloader*100))

        i += 1

    # gpu to cpu
    out_tensor = out_tensor.cpu()
    out_label = out_label.cpu() 

    # append to out_list
    out_list.append(out_tensor)

    # top k accuarcy & append to top_k list
    top_acc = accuracy(out_tensor, out_label, topk=(1,5))
    top_1.append(top_acc[0].item())
    top_5.append(top_acc[1].item())
    
    print('top_acc:', top_acc)

    print('progress: 100% ')

In [ ]:
print(len(out_list))
print(top_1)
print(top_5)

In [ ]:
same_weight_tensor = torch.zeros(out_list[0].size())
top_1_tensor = torch.zeros(out_list[0].size())
top_5_tensor = torch.zeros(out_list[0].size())


for index, item in enumerate(out_list):
    same_weight_tensor += item
    top_1_tensor += top_1[index] * item
    top_5_tensor += top_5[index] * item
   

In [ ]:
accuracy(out_list[0], out_label, topk=(1,))

In [ ]:
accuracy(out_list[1], out_label, topk=(1,))

In [ ]:
accuracy(same_weight_tensor, out_label, topk=(1,))

In [ ]:
accuracy(top_1_tensor, out_label, topk=(1,))

In [ ]:
accuracy(top_5_tensor, out_label, topk=(1,))

In [ ]:
# one-hot encoding to class label
# choose which tensor you want to transform to csv

pred = top_1_tensor.data.max(1, keepdim=True)[1]

In [ ]:
image_list = dataset_t.image_paths

In [ ]:
len(image_list)

In [ ]:
import csv
csv_name = s_name+'_'+t_name+'_pred.csv'
f = open(csv_name,'w') 
w = csv.writer(f)
title = ['image_name','label']
w.writerow(title)
for i in range(len(pred)):
    content = [image_list[i],pred_list[i]]
    w.writerow(content)